In [1]:
from dataclasses import dataclass, field

In [2]:
@dataclass
class QuestionItem:
    question: str
    options: list[str]
    reasoning: str | None = field(default=None)
    answer: str | None = field(default=None)

@dataclass
class Entry:
    problem: str = field(default="")
    questions: list[QuestionItem] = field(default_factory=list)

In [3]:
import pickle
entries = pickle.load(open("./entries.pkl", "rb"))

In [4]:
import dotenv
dotenv.load_dotenv("../.env")

True

In [5]:
[
    e for e in entries if len(e.questions) == 0
]

[]

In [6]:
import re
answer_regex = re.compile(r"答案.*?([A-Z])")
export_entries = []
for entry in entries:
    export_questions = []
    for question in entry.questions:
        # if question.reasoning is None:
        #     continue
        matches = answer_regex.findall(question.reasoning)
        answer = matches[-1] if len(matches) > 0 else None
        # if answer != question.answer:
        #     # print(f"答案不匹配: {answer} != {question.answer}")
        #     continue
        export_questions.append(QuestionItem(
            question=question.question, options=question.options, reasoning=question.reasoning, answer=question.answer
        ))
    if len(export_questions) == 0:
         continue
    export_entries.append(Entry(problem=entry.problem, questions=export_questions))

In [7]:
# deep copy
export_entries_translated = [
    Entry(
        problem=entry.problem,
        questions=[
            QuestionItem(
                question=question.question,
                options=question.options,
                reasoning=question.reasoning,
                answer=question.answer
            )
            for question in entry.questions
        ]
    )
    for entry in export_entries
]

In [8]:
[
    e for e in export_entries_translated if len(e.questions) == 0
]

[]

In [9]:
for entry in export_entries_translated:
    if "翻译" in entry.problem:
        export_entries_translated.remove(entry)

In [10]:
len(export_entries_translated)

280

In [11]:
from tqdm.notebook import tqdm

In [12]:
def translate(text: list[str], trans_back=False) -> list[str]:
    from tencentcloud.common import credential
    from tencentcloud.tmt.v20180321 import tmt_client, models
    import os
    cred = credential.Credential(
        os.getenv("TENCENT_SECRET_ID"),
        os.getenv("TENCENT_SECRET_KEY")
    )
    from tencentcloud.common.profile.client_profile import ClientProfile
    from tencentcloud.common.profile.http_profile import HttpProfile
    httpProfile = HttpProfile()
    httpProfile.endpoint = "tmt.tencentcloudapi.com"
    clientProfile = ClientProfile()
    clientProfile.httpProfile = httpProfile
    client = tmt_client.TmtClient(cred, "ap-beijing", clientProfile)
    request = models.TextTranslateBatchRequest()
    params = {
        "Source": "zh" if not trans_back else "ja",
        "Target": "ja" if not trans_back else "zh",
        "ProjectId": 0,
        "SourceTextList": text
    }
    import json
    request.from_json_string(json.dumps(params))
    response = client.TextTranslateBatch(request)
    return json.loads(response.to_json_string())["TargetTextList"]
    

In [17]:
failed = []
def translate_entries_inplace(entries: list[Entry], trans_back=False):
    try:
        question_flat = [
            question.question
            for entry in entries
            for question in entry.questions
        ]
        reasoning_flat = [
            question.reasoning
            for entry in entries
            for question in entry.questions
        ]
        translated_question_flat = translate(question_flat, trans_back)
        translated_reasoning_flat = translate(reasoning_flat, trans_back)

        i = 0
        for entry in entries:
            for question in entry.questions:
                question.question = translated_question_flat[i]
                question.reasoning = translated_reasoning_flat[i]
                i += 1
        
        problem_flat = [entry.problem for entry in entries]
        translated_problem_flat = translate(problem_flat, trans_back)
        i = 0
        for entry in entries:
            entry.problem = translated_problem_flat[i]
            i += 1
        
        options_flat = [
            option
            for entry in entries
            for question in entry.questions
            for option in question.options
        ]
        translated_options_flat = translate(options_flat, trans_back)
        i = 0
        for entry in entries:
            for question in entry.questions:
                for j in range(len(question.options)):
                    question.options[j] = translated_options_flat[i]
                    i += 1
    except Exception as e:
        failed.extend(entries)
        print(e)

In [14]:
# export_entries_translated = export_entries_translated[:2]

In [15]:
export_entries_translated[:2]

[Entry(problem='在一场山地自行车比赛中，四位选手分别取得了第一、第二、第三和第四名。不同的颜色代表不同的排名。下面是一些关于这场比赛和选手排名的信息：\n\n1. Alan 名列第一。\n2. 第二名的选手穿红色服装。\n3. John 没有穿黄色服装。\n4. 第四名的选手穿蓝色服装。\n5. Steve 的排名和他的服装色是相同的名次。\n6. Kev 的名次排在 Steve 前面。\n7. 第二名的选手穿的不是青色。\n8. 黄色服穿的选手的成绩排在绿色服穿的选手前面。\n9. 确保每四个参数中的所有元素都不相同和符合排名顺序。\n\n根据上述信息， 回答以下选择题：', questions=[QuestionItem(question='选择题 1:\n根据比赛结果，排在第二名之后的可能是哪些名次？', options=['第一名 第三名', '第三名 第四名'], reasoning='好的，让我们一步一步来分析这个问题。\n\n首先，我们知道 Alan 名列第一（信息 1），所以第一名的位置已经确定了。\n\n接下来，我们知道第二名的选手穿红色服装（信息 2）。\n\n然后，我们知道 John 没有穿黄色服装（信息 3），这意味着 John 不是第二名，因为第二名已经给了红色服装。\n\n现在，我们知道第四名的选手穿蓝色服装（信息 4）。\n\n接着，Steve 的排名和他的服装色是相同的名次（信息 5）。这意味着 Steve 的名次要么是红色（第二名），要么是蓝色（第四名），因为这是我们目前知道的唯一两种颜色。\n\n由于 Kev 的名次排在 Steve 前面（信息 6），那么 Kev 只能是第一名或第二名。但是，Alan 已经名列第一（信息 1），所以 Kev 只能是第二名。这意味着 Steve 不能是第二名，因为 Kev 已经占据了第二名。\n\n因此，Steve 只能是第四名（信息 5 和 6），这意味着他穿蓝色服装。\n\n现在我们知道第二名（红色服装）是 Kev，第四名（蓝色服装）是 Steve。\n\n由于黄色服穿的选手的成绩排在绿色服穿的选手前面（信息 8），而我们已经确定了蓝色和红色服装的选手的位置，所以黄色服装的选手只能是第三名，绿色服装的选手只能是第四名。\n\n所以，排在第二名之后的可能是第三名和第四名。\n\n因

In [18]:
batch = 1
for i in tqdm(range(0, len(export_entries_translated), batch)):
    from_idx = i
    to_idx = min(i + batch, len(export_entries_translated))
    translate_entries_inplace(export_entries_translated[from_idx:to_idx])
    translate_entries_inplace(export_entries_translated[from_idx:to_idx], True)

  0%|          | 0/169 [00:00<?, ?it/s]

[TencentCloudSDKException] code:UnsupportedOperation.TextTooLong message:Please ensure that a single request length less than 6000 requestId:b2b22311-412c-49e3-9050-e39f19301591
[TencentCloudSDKException] code:UnsupportedOperation.TextTooLong message:Please ensure that a single request length less than 6000 requestId:b66f92b8-584b-448e-8798-ff3798e94c10


In [28]:
entries[:3]

[Entry(problem='有一个英文到法文的词汇表，包含以下对应词汇：\n\n1. the -> le\n2. cat -> chat\n3. jumps -> sauts\n4. over -> sur\n5. moon -> lune\n6. cow -> vache\n7. plays -> jouer\n8. fiddle -> violon\n9. egg -> bougre\n10. falls -> des chutes\n11. off -> de\n12. wall -> mur\n\n根据这个词汇表，翻译以下英文句子成法文：', questions=[QuestionItem(question='选择题 1：\n英文句子 "the cat jumps over the moon" 翻译成法文是：', options=['le chat saute sur la lune', 'le chat sauts sur le lune', 'le sauts chat sur le lune', 'le chat sauts sur le lune'], reasoning=' 首先，我们根据英文到法文的词汇表来翻译句子中的每个单词：\n\n1. "the" -> "le"，所以 "the cat" 变成 "le chat"。\n2. "cat" -> "chat"，所以 "cat" 保持不变，依然是 "chat"。\n3. "jumps" -> "sauts"，所以 "jumps over" 变成 "sauts sur"。\n4. "over" -> "sur"，所以 "over the moon" 变成 "sur la lune"。\n\n综上所述，根据词汇表翻译，英文句子 "the cat jumps over the moon" 变成法文是 "le chat sauts sur la lune"。\n\n选项A："le chat saute sur la lune" 错误，因为 "jumps" 对应 "sauts" 而不是 "sautte"。\n选项B："le chat sauts sur le lune" 错误，因为 "over the moon" 应该用 "sur la lune" 而不是 "sur le lune"。\n选项C："le

In [29]:
export_entries_translated[:2]

[Entry(problem='在山地自行车比赛中，四名选手分别获得第一名、第二名、第三和第四名。不同的颜色代表不同的等级。以下是有关游戏和玩家排名的信息。\n\n艾伦是第一位的。\n2.第二名的球员穿红色的衣服。\n3.约翰没有穿黄色的衣服。\n4.第四名的球员将穿着蓝色的衣服。\n5.史蒂夫的排名和他的衣服颜色是一样的\n“史蒂夫站在史蒂夫的前面。\n7.排名第二的球员没有穿蓝色的衣服。\n8.穿了黄色衣服的，在穿了绿色衣服的上位。\n9.确保这四个参数中的所有元素都不是相同的，并且是按顺序排列的。\n\n根据上述内容，回答以下的选择。', questions=[QuestionItem(question='第一点选择。\n根据结果，第二名之后的排名会是什么？', options=['第三，第三，第三', '第三，第四，'], reasoning='好的，让我们一步一步地分析这个问题。\n\n首先，由于可知Alan是第1位(信息1)，所以决定了第1位的位置。\n\n其次，可以看出排在第二位的选手穿着红色服装(信息2)。\n\n而且，我们知道John没有穿黄色衣服(消息3)，这意味着John不是第二名。\n\n据了解，目前排名第四的选手穿着蓝色服装(信息4)。\n\n其次，Steve的排名与他的着装颜色排名相同(信息5)。这意味着Steve的排名是红色(第二名)还是蓝色(第四名)。因为这是我们现在知道的唯一两种颜色。\n\n由于Kev的排名排在Steve的前列(信息6)，Kev只能排在第一位或第二位。但是，由于Alan已经排在第一位(消息1)，Kev只能排在第二位。这意味着Kev不能排在第二位，因为Steve占据第二位。\n\n史蒂夫只有第四名(信息5和6)，这意味着他们穿着蓝色衣服。\n\n现在我们知道第二名(红色衣服)是Kev，第四名(蓝色衣服)是Steve。\n\n穿黄色衣服的选手成绩排在穿绿色衣服的选手前面(信息8)，但是因为穿着蓝色和红色衣服的选手的位置已经确定了，所以黄色衣服的选手只排在第3名，绿色衣服的选手排在第4名。\n\n因此，第2位以后有可能成为第3位和第4位。\n\n因此，答案是B：第3位第4位。', answer='B'), QuestionItem(question='选择的第二点。\n第一个在第三个之前？', options=['是的，

In [30]:
pickle.dump(
    export_entries_translated, open("./entries_aug.pkl", "wb")
)